# 飞桨常规赛：[遥感影像地块分割](https://aistudio.baidu.com/aistudio/competition/detail/63) - 4月第10名方案

## 0 引言

最近一段时间学习了飞桨领航团实战速成营课程，对图像分割有了一定的了解了，课程中有一个非常有趣的实践作业——参加飞桨常规赛：遥感影像地块分割。

在作业中笔者分别使用：
- FCN（Fully Convolutional Networks）网络
- U-Net网络
- DeepLabV3P网络
以上三种网络模型对遥感影像地块分割进行实验，具体内容详见下文。

**下面的表格是课程地址和|PaddleSeg代码地址：**
|标题|链接|
|-|-|
|飞桨领航团实战速成营课程|[链接](https://aistudio.baidu.com/aistudio/education/group/info/16606/topicdirectly=1&shared=1)|
|图像分割理论综述学习笔记|[链接](https://aistudio.baidu.com/aistudio/projectdetail/1782201)|
|PaddleSeg的github地址|[链接](https://github.com/PaddlePaddle/PaddleSeg)|
|PaddleSeg的gitee地址|[链接](https://gitee.com/paddlepaddle/PaddleSeg)|

## 1 比赛介绍

### 1.1 赛题介绍
本赛题由 2020 CCF BDCI 遥感影像地块分割 初赛赛题改编而来。遥感影像地块分割, 旨在对遥感影像进行像素级内容解析，对遥感影像中感兴趣的类别进行提取和分类，在城乡规划、防汛救灾等领域具有很高的实用价值，在工业界也受到了广泛关注。现有的遥感影像地块分割数据处理方法局限于特定的场景和特定的数据来源，且精度无法满足需求。因此在实际应用中，仍然大量依赖于人工处理，需要消耗大量的人力、物力、财力。本赛题旨在衡量遥感影像地块分割模型在多个类别（如建筑、道路、林地等）上的效果，利用人工智能技术，对多来源、多场景的异构遥感影像数据进行充分挖掘，打造高效、实用的算法，提高遥感影像的分析提取能力。

### 1.2 赛题任务
本赛题旨在对遥感影像进行像素级内容解析，并对遥感影像中感兴趣的类别进行提取和分类，以衡量遥感影像地块分割模型在多个类别（如建筑、道路、林地等）上的效果。

### 1.3 数据说明
本赛题提供了多个地区已脱敏的遥感影像数据，各参赛选手可以基于这些数据构建自己的地块分割模型。

### 1.4 训练数据集
样例图片及其标注如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/96da8ef3e3d54f14b202d6c6184c71aa8e307447450d4d6492326138089579f8)

训练数据集文件名称：train_and_label.zip
包含2个子文件，分别为：训练数据集（原始图片）文件、训练数据集（标注图片）文件，详细介绍如下：

- 训练数据集（原始图片）文件名称：img_train
	包含66,653张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，每张图片的名称形如T000123.jpg。

- 训练数据集（标注图片）文件名称：lab_train
	包含66,653张分辨率为2m/pixel，尺寸为256 * 256的PNG图片，每张图片的名称形如T000123.png。
    
  **备注：**  全部PNG图片共包括4种分类，像素值分别为0、1、2、3。此外，像素值255为未标注区域，表示对应区域的所属类别并不确定，在评测中也不会考虑这部分区域。

### 1.5 测试数据集
测试数据集文件名称：img_test.zip，详细介绍如下：
包含4,609张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，文件名称形如123.jpg。

## 2. 准备下载相关库与数据解压

In [ ]:
#下载PaddleSeg
!git clone https://gitee.com/paddlepaddle/PaddleSeg.git

Cloning into 'PaddleSeg'...
remote: Enumerating objects: 10924, done.
remote: Counting objects: 100% (10924/10924), done.
remote: Compressing objects: 100% (5412/5412), done.
remote: Total 10924 (delta 7389), reused 8171 (delta 5353), pack-reused 0
Receiving objects: 100% (10924/10924), 156.94 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (7389/7389), done.
Checking connectivity... done.


In [ ]:
#将赛题的数据解压缩到用户目录~/
!unzip -q data/data77571/train_and_label.zip > /dev/null
!unzip -q data/data77571/img_test.zip > /dev/null

In [ ]:
!rm -r __MACOSX/

## 3. 数据标签分析

In [ ]:
#测试输出图片
pic1 = '/home/aistudio/lab_train/T020005.png'
from PIL import Image
import numpy as np
img = Image.open(pic1)
test  = np.array(img)
# print(test.shape)
a = test.flatten()
set(a.tolist())

{1, 255}

根据上面一张图片的结果可见，图片中像素值只有1代表一种类别和255代表没有标注的区域，和赛题描述的情况一致。

### 3.1 数据类型分布的分析
<font size="5pt" color="red">注意：下面这部分数据分析的代码参考[坑姐](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/90149)的[项目](https://aistudio.baidu.com/aistudio/projectdetail/1752986)，这部分数据分析的代码可以用于任何类似的任务，非常有学习的价值！</font>

In [ ]:
import cv2, os
import numpy as np

#amount of classer
CLASSES_NUM = 4

#find imagee in folder dir
def findImages(dir,topdown=True):
    im_list = []
    if not os.path.exists(dir):
        print("Path for {} not exist!".format(dir))
        raise
    else:
        for root, dirs, files in os.walk(dir, topdown):
            for fl in files:
                im_list.append(fl)
    return im_list

# amount of images corresponding to each classes
images_count = [0]*CLASSES_NUM
# amount of pixels corresponding to each class
class_pixels_count = [0]*CLASSES_NUM
# amount of pixels corresponding to the images of each class
image_pixels_count = [0]*CLASSES_NUM

image_folder = './lab_train/'
im_list = findImages(image_folder) 

In [ ]:
#######################################################注意！！！！！！！！！！！
# 这部分代码没事就不要再次运行了，非常消耗时间！！
for im in im_list:
    # print(im)
    cv_img = cv2.imread(os.path.join(image_folder, im), cv2.IMREAD_UNCHANGED)
    size_img = cv_img.shape
    colors = set([])
    for i in range(size_img[0]):
        for j in range(size_img[1]):
            p_value = cv_img.item(i,j)
            if not p_value < CLASSES_NUM: # check
                pass
                # print(p_value) #因为有255，所以不打印了
            else:
                class_pixels_count[p_value] = class_pixels_count[p_value] + 1
                colors.add(p_value)
    im_size = size_img[0]*size_img[1]
    for n in range(CLASSES_NUM):
        if n in colors:
            images_count[n] = images_count[n] + 1
            image_pixels_count[n] = image_pixels_count[n] + im_size

print(images_count)
print(class_pixels_count)
print(image_pixels_count)

[23707, 30151, 23044, 18978]
[247142772, 310655521, 152480301, 159504650]
[1553661952, 1975975936, 1510211584, 1243742208]


In [ ]:
# t = class_pixels_count.copy()
t = [247142772, 310655521, 152480301, 159504650]
a = np.array(t).sum()
for i in range(len(t)):
    t[i] = round(t[i]/a, 4)
print(t)

[0.2841, 0.3572, 0.1753, 0.1834]


<font color="red" size="5pt">小结：根据打印结果可知，可以看出类别比较均衡</font>

### 3.2 数据准备（生成用于训练得到标签文件）

In [ ]:
#首先进入到用户目录
%cd ~

/home/aistudio


In [ ]:
import os
from imghdr import what
import numpy as np
import random

def get_all_date(dir_images, dir_masks):
    """生成训练、测试所需的txt文件"""
    res = []
    img_list = os.listdir(dir_images)
    mask_list = os.listdir(dir_masks)
    assert len(img_list) == len(mask_list)  #样本和标签数量肯定要一样
    img_suffix = os.path.splitext(os.path.join(dir_images,img_list[0]))[1]
    mask_suffix = os.path.splitext(os.path.join(dir_masks,mask_list[0]))[1]
    print(f'The suffix of images is: {img_suffix} \n The suffix of masks is {mask_suffix}!')
    for i in range(len(img_list)):
        img_name = img_list[i]
        mask_name = os.path.splitext(img_name)[0]+mask_suffix
        image_path = os.path.join(dir_images,img_name)
        mask_path = os.path.join(dir_masks,mask_name)
        if os.path.splitext(img_name)[0] == os.path.splitext(mask_name)[0] and what(image_path) and what(mask_path): 
            #样本和标签文件名一样（不包含后缀，因为后缀可能不一样），同时都为图片
            res.append((image_path, mask_path))
        else:
            print(image_path, mask_path)
    return res

res = get_all_date('img_train', 'lab_train')
print(f'The samples of image and label is {res[:3]}')
print(f'The effective number of image data is: {len(res)}')
random.shuffle(res)
random.shuffle(res)
pro = 0.95 #训练/验证比例
pro_int = int(len(res)*pro)
print(f'The split number of train data is: {pro_int}')
with open('./train_list.txt', 'w') as f:
    for line in res[:pro_int]:
        f.writelines(line[0] + ' ' + line[1] + '\n')

with open('./val_list.txt', 'w') as f:
    for line in res[pro_int:]:
        f.writelines(line[0] + ' ' + line[1] + '\n')

The suffix of images is: .jpg 
 The suffix of masks is .png!
The samples of image and label is [('img_train/T057389.jpg', 'lab_train/T057389.png'), ('img_train/T040500.jpg', 'lab_train/T040500.png'), ('img_train/T039138.jpg', 'lab_train/T039138.png')]
The effective number of image data is: 66652
The split number of train data is: 63319


## 4. 修改配置文件并进行训练

根据[图像分割理论综述学习笔记](https://aistudio.baidu.com/aistudio/projectdetail/1782201)中介绍可以看出，杨老师原始的[课程视频](https://www.bilibili.com/video/BV1pp4y1871g)中主要介绍了五种典型的图像分割网络：
- FCN（Fully Convolutional Networks）网络
- U-Net网络
- DeepLabV3P网络
- HRNet网络
- OCRNet网络
各种网络中的具体介绍详见[图像分割理论综述学习笔记](https://aistudio.baidu.com/aistudio/projectdetail/1782201)。本项目使用其中的前三种网络进行实验

In [ ]:
#进入PaddleSeg目录
%cd ./PaddleSeg/

/home/aistudio/PaddleSeg


### 4.1 实验一：使用“FCN（Fully Convolutional Networks）网络”进行训练
<font size="4">根据[自定义数据集文档](https://gitee.com/paddlepaddle/PaddleSeg/blob/release/v2.0/docs/data_prepare.md)自己继承并实现配置文件`/home/aistudio/My_Config/My_fcn.yml`，运行下面的命令进行训练</font>

#### （1）进行训练

In [8]:
#注意：evaluate会在每次保存save_interval的时候进行一次，并且于历史数据对比，如果精度提高则更新best_model
!python train.py \
       --config /home/aistudio/My_Config/My_fcn.yml \
       --do_eval \
       --use_vdl \
       --save_interval 250 \
       --save_dir /home/aistudio/output_myfcn

注意，因为训练结果打印的日志太长了，这里将其删除，最终的输出最佳结果文件在<font color="red">/home/aistudio/output_myfcn/best_model</font>文件夹内

#### （2）使用训练得到的最佳模型进行评估

In [16]:
!python val.py \
       --config /home/aistudio/My_Config/My_fcn.yml \
       --model_path /home/aistudio/output_myfcn/best_model/model.pdparams
#输出结果：Images=3333 mIoU=0.4754 Acc=0.6818 Kappa=0.5481

### 4.2 实验二：使用“U-Net网络”进行训练
<font size="4">根据[自定义数据集文档](https://gitee.com/paddlepaddle/PaddleSeg/blob/release/v2.0/docs/data_prepare.md)自己继承并实现配置文件`/home/aistudio/My_Config/My_u2net.yml`，运行下面的命令进行训练</font>

#### （1）进行训练

In [ ]:
!python train.py \
       --config /home/aistudio/My_Config/My_u2net.yml \
       --do_eval \
       --use_vdl \
       --save_interval 250 \
       --save_dir /home/aistudio/output_myu2net

注意，因为训练结果打印的日志太长了，这里将其删除，最终的输出最佳结果文件在<font color="red">/home/aistudio/output_myu2net/best_model</font>文件夹内

#### （2）使用训练得到的最佳模型进行评估

In [ ]:
!python val.py \
       --config /home/aistudio/My_Config/My_u2net.yml \
       --model_path /home/aistudio/output_myu2net/best_model/model.pdparams
#输出结果：Images=3333 mIoU=0.1860 Acc=0.4547 Kappa=0.1766 

### 4.3 实验三：使用“DeepLabV3P网络”进行训练
<font size="4">根据[自定义数据集文档](https://gitee.com/paddlepaddle/PaddleSeg/blob/release/v2.0/docs/data_prepare.md)自己继承并实现配置文件`/home/aistudio/My_Config/My_deep.yml`，运行下面的命令进行训练</font>

#### （1）进行训练

In [ ]:
!python train.py \
       --config /home/aistudio/My_Config/My_deep.yml \
       --do_eval \
       --use_vdl \
       --save_interval 250 \
       --save_dir /home/aistudio/output_mydeep

In [ ]:
# 从某一轮继续训练的命令
# !python train.py \
#        --config /home/aistudio/My_Config/My_deep.yml \
#        --resume_model output_deeplabv3/iter_XXX \
#        --do_eval \
#        --use_vdl \
#        --save_interval 750 \
#        --save_dir /home/aistudio/output_mydeep

注意，因为训练结果打印的日志太长了，这里将其删除，最终的输出最佳结果文件在<font color="red">/home/aistudio/output_mydeep/best_model</font>文件夹内

#### （2）使用训练得到的最佳模型进行评估

In [21]:
!python val.py \
       --config /home/aistudio/My_Config/My_deep.yml \
       --model_path /home/aistudio/output_mydeep/best_model/model.pdparams
#输出结果：Images=3333 mIoU=0.6166 Acc=0.7802 Kappa=0.6952 

### 4.4 实验结果

| 名称 | 方案 | 迭代次数 | mIoU | Acc | 参数路径 |
|---|---|---|---|---|---|
| 实验一  | 使用“FCN（Fully Convolutional Networks）网络”进行训练 | 2000(第一次没有保存结果，重新训练为了节约时间，提前终止了训练) | 0.4754 | 0.6818|/home/aistudio/output_myfcn/best_model/model.pdparams|
| 实验二  | 使用“U-Net网络”进行训练 | 10000 | 0.1860 | 0.4547|/home/aistudio/output_myu2net/best_model/model.pdparams|
| 实验三  | 使用“DeepLabV3P网络”进行训练 | 50000 | 0.6166 |0.7802|/home/aistudio/output_mydeep/best_model/model.pdparams|

<font size="6px" color="red">注意：上述模型预训练参数保存在项目中，因此需要自己fork并运行该项目，然后进入到环境中得到。</font>


## 5. 预测
根据上述实验结果，选择<font color="red">实验三的参数</font>进行预测

PS(postscript，补充一下)：PaddleSeg比较贴心，默认训练的时候会保存最佳的结果，希望以后官方的其他库也能实现这个功能

In [20]:
!python predict.py \
       --config /home/aistudio/My_Config/My_deep.yml \
       --model_path /home/aistudio/output_mydeep/best_model/model.pdparams \
       --image_path ../img_testA/ \
       --save_dir ../submit/

<font size="4pt">最后将`~/submit/`下面的`pseudo_color_prediction`文件夹重命名为`resutl`，该文件夹就是最终的结果文件夹，里面是分割后的预测图片</font>

## 6. 总结
- 1 FCN,U-Net,DeepLabV3P网络相比，DeepLabV3P效果最好，当然很有可能由于DeepLabV3P训练次数最多，有兴趣的同学可以试试训练更多的次数进行实验
- 2 后续可以使用其他网络进行实验，并且可以调整模型参数，这样可能会达到更佳预测的精度

# 个人介绍

<font size="6px" color="red">我在AI Studio上获得黄金等级，点亮6个徽章，来互关呀~</font> <font size="6px" color="red">[https://aistudio.baidu.com/aistudio/personalcenter/thirdview/398864](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/398864)</font>

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 